In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras import layers
from keras.layers import LSTM
from keras import backend as K 
from sklearn.metrics import mean_absolute_error
 
x_meanf=[None] *30
x_stdf = [None] *30
y_meanf=[None] *30
y_stdf =[None] * 30
val_mae_end = [None] * 30
mae_end = [None]  * 30
correl = [None]  * 30
mae_actual = [None]  * 30
correl_actual = [None]  * 30
for l in range(0,30) : 

        df = pd.read_csv('out_lead_%dday_data.csv'%(l),delimiter = '\t')

        df=df.sample(frac=1).reset_index(drop=True)

        val_mae = [None] * 5
        mae = [None] * 5
    
        df.iloc[:,16] =abs(df.iloc[:,16] - df.iloc[:,15])
        
        x, y = df.iloc[:, 1:16], df.iloc[:,16]

        y = y.values.reshape(y.shape[0],1)
        x_train, x_test, y_train, y_test = \
        train_test_split(x, y, test_size=0.1, 
                                         random_state=0)
        x_mean = x_train.mean(axis=0)
        x_train -= x_mean
        x_std = x_train.std(axis=0)
        x_train /= x_std

        x_test -= x_mean
        x_test /= x_std 

        y_mean = y_train.mean(axis=0)
        y_train -=y_mean
        y_std=y_train.std(axis=0)
        y_train /=y_std

        y_test -=y_mean
        y_test /=y_std
        
        x_meanf[l] = x_mean
        x_stdf[l]  = x_std
        y_meanf[l] = y_mean
        y_stdf[l]  = y_std
        
        import time
        # Measuring the time taken by the model to train
        StartTime=time.time()
            # Initialising the RNN
        model= Sequential()

            # Adding the First input hidden layer and the LSTM layer
            # return_sequences = True, means the output of every time step to be shared with hidden next layer
        model.add(layers.Dense(2, activation='relu',
        input_shape=(x_train.shape[1],)))
        model.add(layers.Dense(2, activation='relu'))
        #model.add(layers.Dropout(0.5))
        model.add(layers.Dense(1))
        model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])


                # Fitting the RNN to the Training set
        history=model.fit(x_train, y_train,validation_data=(x_test, y_test), batch_size = 1000, epochs = 12)
            #history=model.fit(x_train, y_train, batch_size = 1000, epochs = 20) 3
        
        model.save('my_model_%d_day.h5'%(l))
           
        del model
 
        
        '''
        mae_actual[l] = mean_absolute_error(df["ModelFlow"] , df["Obsflow"] )
        correl_actual[l] = df["ModelFlow"].corr(df["Obsflow"])
        val_mae_end[l]=min(val_mae)
        a =model.predict(x_test)
        b= y_test
        d_a= pd.DataFrame(a)
        d_b=pd.DataFrame(b)
        d_ab=pd.concat([d_a, d_b], axis=1)
        d_ab.columns = ['predicted', 'actual']
        d_ab['index'] = d_ab.index.values
        #d_ab.to_csv('out_%d_day_predicted_data.csv'%(l))
        correl[l] = d_ab['predicted'].corr(d_ab['actual'])
        mae_end[l]=min(mae)
        
        '''

        

Epoch 1/12
139/139 [==============================] - 1s 2ms/step - loss: 1.0315 - mae: 0.3460 - val_loss: 0.9277 - val_mae: 0.2974
Epoch 2/12
139/139 [==============================] - 0s 1ms/step - loss: 0.8786 - mae: 0.2749 - val_loss: 0.8155 - val_mae: 0.2546
Epoch 3/12
139/139 [==============================] - 0s 1ms/step - loss: 0.7504 - mae: 0.2397 - val_loss: 0.6706 - val_mae: 0.2255
Epoch 4/12
139/139 [==============================] - 0s 1ms/step - loss: 0.6411 - mae: 0.2120 - val_loss: 0.5744 - val_mae: 0.2051
Epoch 5/12
139/139 [==============================] - 0s 1ms/step - loss: 0.5882 - mae: 0.1949 - val_loss: 0.5454 - val_mae: 0.1899
Epoch 6/12
139/139 [==============================] - 0s 1ms/step - loss: 0.5691 - mae: 0.1856 - val_loss: 0.5354 - val_mae: 0.1799
Epoch 7/12
139/139 [==============================] - 0s 1ms/step - loss: 0.5631 - mae: 0.1786 - val_loss: 0.5319 - val_mae: 0.1746
Epoch 8/12
139/139 [==============================] - 0s 1ms/step - loss: 0.

In [2]:
d_x= pd.DataFrame(x_meanf)
d_y=pd.DataFrame(x_stdf)
d_z=pd.DataFrame(y_meanf)
d_t=pd.DataFrame(y_stdf)
d_xy=pd.concat([d_x, d_y,d_z,d_t], axis=1)
d_xy.columns = ['x_mean', 'x_stdf','y_meanf','y_stdf']
d_xy
d_xy.to_csv("x-y_meanstd.csv")

ValueError: Length mismatch: Expected axis has 32 elements, new values have 4 elements

In [3]:
d_x.to_csv("x_mean.csv", sep='\t')
d_y.to_csv("x_std.csv", sep='\t')
d_z.to_csv("y_mean.csv",  sep='\t')
d_t.to_csv("y_std.csv", sep='\t')